# Prerrequisites

Installing Spark and Apache Kafka Library in VM


---



In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark==1.3.0
!pip install py4j==0.10.9

# For plotting
!pip install folium
!pip install plotly

     |████████████████████████████████| 204kB 8.8MB/s 


In [ ]:
!ls /content

sample_data  spark-3.0.1-bin-hadoop2.7	spark-3.0.1-bin-hadoop2.7.tgz


In [ ]:
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/libs/libs-kafka_301.zip --directory-prefix=/content/spark-3.0.1-bin-hadoop2.7/jars/
!unzip -n /content/spark-3.0.1-bin-hadoop2.7/jars/libs-kafka_301.zip -d /content/spark-3.0.1-bin-hadoop2.7/jars/
!ls /content/spark-3.0.1-bin-hadoop2.7/jars/*kafka*

Archive:  /content/spark-3.0.1-bin-hadoop2.7/jars/libs-kafka_301.zip
  inflating: /content/spark-3.0.1-bin-hadoop2.7/jars/ant-1.10.3.jar  
  inflating: /content/spark-3.0.1-bin-hadoop2.7/jars/ant-launcher-1.10.3.jar  
  inflating: /content/spark-3.0.1-bin-hadoop2.7/jars/asm-7.0.jar  
  inflating: /content/spark-3.0.1-bin-hadoop2.7/jars/asm-tree-7.0.jar  
  inflating: /content/spark-3.0.1-bin-hadoop2.7/jars/asm-util-7.0.jar  
  inflating: /content/spark-3.0.1-bin-hadoop2.7/jars/cglib-3.2.10.jar  
  inflating: /content/spark-3.0.1-bin-hadoop2.7/jars/commons-pool2-2.6.2.jar  
  inflating: /content/spark-3.0.1-bin-hadoop2.7/jars/kafka-clients-2.4.1.jar  
  inflating: /content/spark-3.0.1-bin-hadoop2.7/jars/spark-sql-kafka-0-10_2.12-3.0.1.jar  
  inflating: /content/spark-3.0.1-bin-hadoop2.7/jars/spark-token-provider-kafka-0-10_2.12-3.0.1.jar  
/content/spark-3.0.1-bin-hadoop2.7/jars/kafka-clients-2.4.1.jar
/content/spark-3.0.1-bin-hadoop2.7/jars/libs-kafka_301.zip
/content/spark-3.0.1-bin-

Define the environment (Java & Spark homes)

---

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"
os.environ["PYSPARK_SUBMIT_ARGS"] = ""

Starting Spark Session and print the version


---


In [ ]:
import findspark
findspark.add_packages(["org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1"])
findspark.add_jars(["/content/spark-3.0.1-bin-hadoop2.7/jars/kafka-clients-2.0.0.jar","/content/spark-3.0.1-bin-hadoop2.7/jars/lz4-java-1.4.1-jar","/content/spark-3.0.1-bin-hadoop2.7/jars/scala-library-2.11.12.jar","/content/spark-3.0.1-bin-hadoop2.7/jars/slf4j-api-1.7.25.jar","/content/spark-3.0.1-bin-hadoop2.7/jars/snappy-java-1.1.7.1.jar","/content/spark-3.0.1-bin-hadoop2.7/jars/spark-sql-kafka-0-10_2.11-2.4.5.jar","/content/spark-3.0.1-bin-hadoop2.7/jars/spark-tags_2.11-2.4.5.jar","/content/spark-3.0.1-bin-hadoop2.7/jars/unused-1.0.0.jar"])
findspark.init("spark-3.0.1-bin-hadoop2.7")# SPARK_HOME

from pyspark.sql import SparkSession

# create the session
spark = SparkSession \
        .builder \
        .master("local[*]") \
        .config("spark.ui.port", "4050") \
        .getOrCreate()

spark.version

'3.0.1'

In [ ]:
spark

In [ ]:
# For Pandas conversion optimization
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

Creating ngrok tunnel to allow Spark UI (Optional)


In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2021-01-25 12:29:29--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.6.49.248, 52.2.242.115, 34.196.37.54, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.6.49.248|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  16.9MB/s    in 0.8s    

2021-01-25 12:29:30 (16.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


# Download Datasets

In [ ]:
!mkdir -p /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/trades.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/trades.json -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/offshore_leaks.edges.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/offshore_leaks.nodes.address.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/offshore_leaks.nodes.intermediary.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/offshore_leaks.nodes.officer.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/offshore_leaks.nodes.entity.csv -P /dataset
!ls /dataset

offshore_leaks.edges.csv		 offshore_leaks.nodes.officer.csv.1
offshore_leaks.edges.csv.1		 offshore-leaks.sqlite
offshore_leaks.nodes.address.csv	 trades.csv
offshore_leaks.nodes.address.csv.1	 trades.csv.1
offshore_leaks.nodes.entity.csv		 trades.csv.2
offshore_leaks.nodes.intermediary.csv	 trades.json
offshore_leaks.nodes.intermediary.csv.1  trades.json.1
offshore_leaks.nodes.officer.csv	 trades.json.2


# Project 1 - Regulatory Banking Project
---

Input files: /dataset/trades.csv & /dataset/trades.json


In [ ]:
from pyspark.sql.functions import *

def pnl(open, close):
  if (open is not None and close is not None):
    return (close * 100 / open) - 100
  else:
    return 0

spark.udf.register("pnl_udf", pnl)

<function __main__.pnl>

In [ ]:
df_trade1 = spark \
  .read \
  .format("csv") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/dataset/trades.csv")
  

In [ ]:
df_trade1.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: double (nullable = true)
 |-- Dividend: double (nullable = true)
 |-- Split: double (nullable = true)
 |-- Adj_Open: double (nullable = true)
 |-- Adj_High: double (nullable = true)
 |-- Adj_Low: double (nullable = true)
 |-- Adj_Close: double (nullable = true)
 |-- Adj_Volume: double (nullable = true)



In [ ]:
df_trade1_norm = (df_trade1
                  .select("date", "open", "close")
                  .withColumn("type", lit("trade_1"))
)

df_trade1_norm.show()

+----------+-------+------+-------+
|      date|   open| close|   type|
+----------+-------+------+-------+
|2017-12-28| 190.91|189.78|trade_1|
|2017-12-27|  190.6|190.19|trade_1|
|2017-12-26| 188.53|190.36|trade_1|
|2017-12-22|  188.2|188.13|trade_1|
|2017-12-21|  187.7|188.08|trade_1|
|2017-12-20| 187.14|187.31|trade_1|
|2017-12-19|185.235|185.98|trade_1|
|2017-12-18| 183.96|184.73|trade_1|
|2017-12-15|183.005|182.58|trade_1|
|2017-12-14| 183.88|182.13|trade_1|
|2017-12-13| 182.01|183.03|trade_1|
|2017-12-12| 182.75| 181.8|trade_1|
|2017-12-11|  182.9|182.25|trade_1|
|2017-12-08|  182.5|183.41|trade_1|
|2017-12-07| 180.05| 182.0|trade_1|
|2017-12-06| 180.25| 180.8|trade_1|
|2017-12-05| 184.79|182.85|trade_1|
|2017-12-04| 183.19| 184.9|trade_1|
|2017-12-01| 180.32|180.42|trade_1|
|2017-11-30| 178.07|179.82|trade_1|
+----------+-------+------+-------+
only showing top 20 rows



In [ ]:
df_trade2 = spark \
  .read \
  .json("/dataset/trades.json")

In [ ]:
df_trade2.printSchema()

root
 |-- dataset: struct (nullable = true)
 |    |-- collapse: string (nullable = true)
 |    |-- column_index: string (nullable = true)
 |    |-- column_names: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- data: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |-- database_code: string (nullable = true)
 |    |-- database_id: long (nullable = true)
 |    |-- dataset_code: string (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- end_date: string (nullable = true)
 |    |-- frequency: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- limit: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- newest_available_date: string (nullable = true)
 |    |-- oldest_available_date: string (nullable = true)
 |    |-- order: string (nullable = true)
 |    |-- premium: boolean (nullable = true)
 |  

In [ ]:
df_trade2.select("dataset.column_names").show(truncate= False)

+-----------------------------------------------------------------------------------------------------------+
|column_names                                                                                               |
+-----------------------------------------------------------------------------------------------------------+
|[Date, Open, High, Low, Close, Volume, Dividend, Split, Adj_Open, Adj_High, Adj_Low, Adj_Close, Adj_Volume]|
+-----------------------------------------------------------------------------------------------------------+



In [ ]:
from pyspark.sql.functions import *

df_trade2_norm = df_trade2 \
  .select(explode("dataset.data").alias("data")) \
  .select(element_at("data", 1).alias("date"), element_at("data", 2).alias("open"), element_at("data", 5).alias("close")) \
  .withColumn("type", lit("trade_2"))

In [ ]:
df_trade2_norm.show()

+----------+-------+------+-------+
|      date|   open| close|   type|
+----------+-------+------+-------+
|2017-12-28|  108.0|107.77|trade_2|
|2017-12-27| 108.42|107.64|trade_2|
|2017-12-26| 108.49|108.12|trade_2|
|2017-12-22|  109.4|108.67|trade_2|
|2017-12-21| 109.52|109.57|trade_2|
|2017-12-20|111.625|109.69|trade_2|
|2017-12-19| 111.05|111.81|trade_2|
|2017-12-18| 111.85|111.03|trade_2|
|2017-12-15|111.805|111.27|trade_2|
|2017-12-14| 107.75|110.57|trade_2|
|2017-12-13| 108.15|107.61|trade_2|
|2017-12-12|  107.6|107.43|trade_2|
|2017-12-11| 104.87|106.83|trade_2|
|2017-12-08| 104.91|104.23|trade_2|
|2017-12-07| 105.74|105.26|trade_2|
|2017-12-06| 107.17|105.46|trade_2|
|2017-12-05| 109.53|107.22|trade_2|
|2017-12-04|  106.5|110.22|trade_2|
|2017-12-01| 104.58|105.25|trade_2|
|2017-11-30| 105.36|104.82|trade_2|
+----------+-------+------+-------+
only showing top 20 rows



In [ ]:
from pyspark.sql.window import Window

windowSpec  = Window.partitionBy("date").orderBy(col("pnl").desc())

In [ ]:
df_result = (df_trade1_norm
 .union(df_trade2_norm)
 .withColumn("pnl", round(pnl(col("open"), col("close")), 2))
 .withColumn("row_number", row_number().over(windowSpec)) 
 .filter(col("row_number") == 1)
 .select("date", "pnl", "type")
)

In [ ]:
df_result.show()

+----------+-----+-------+
|      date|  pnl|   type|
+----------+-----+-------+
|2014-05-27| 0.39|trade_1|
|2015-05-01|  2.4|trade_1|
|2016-08-17| 0.48|trade_1|
|2017-12-05|-1.05|trade_1|
|2014-07-14|-0.58|trade_1|
|2014-12-11|  1.0|trade_1|
|2015-02-09|-0.07|trade_2|
|2015-02-27|-0.38|trade_1|
|2016-04-22| 0.69|trade_2|
|2016-08-08|-0.37|trade_2|
|2017-02-24|  0.9|trade_1|
|2017-05-11|-0.15|trade_2|
|2017-10-20|-0.05|trade_2|
|2017-12-22|-0.04|trade_1|
|2015-02-26|-0.53|trade_1|
|2015-11-20| 2.24|trade_1|
|2016-07-06| 1.83|trade_1|
|2013-11-08| 2.56|trade_2|
|2014-03-17| 1.11|trade_2|
|2014-03-20| 0.48|trade_1|
+----------+-----+-------+
only showing top 20 rows



# Project 2 - Transactions Notifications

*Hint: https://databricks.com/blog/2017/05/08/event-time-aggregation-watermarking-apache-sparks-structured-streaming.html*

In [ ]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType

df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "ec2-3-231-22-58.compute-1.amazonaws.com:9092") \
  .option("subscribe", "transactions") \
  .load()

In [ ]:
schema = StructType(
    [
     StructField('Account No', StringType(), True),
     StructField('DATE', StringType(), True),
     StructField('TRANSACTION DETAILS', StringType(), True),
     StructField('CHQ.NO.', StringType(), True),
     StructField('VALUE DATE', StringType(), True),
     StructField(' WITHDRAWAL AMT ', StringType(), True),
     StructField(' DEPOSIT AMT ', StringType(), True),
     StructField('BALANCE AMT', StringType(), True)
    ]
)
df.printSchema()

dataset = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)", "timestamp") \
    .withColumn("value", from_json("value", schema)) \
    .select(col('key'), col("timestamp"), col('value.*'))

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [ ]:
from pyspark.sql.functions import *

dataset_count = \
  dataset \
    .groupBy(window("timestamp", "1 minute")) \
    .count()

In [ ]:
dataset_count.writeStream \
 .outputMode("update") \
 .format("memory") \
 .option("truncate", "false") \
 .queryName("transactions") \
 .start()

In [ ]:
spark.sql("select * from transactions").show(truncate = False)

+------------------------------------------+-----+
|window                                    |count|
+------------------------------------------+-----+
|[2021-01-24 19:10:00, 2021-01-24 19:11:00]|171  |
|[2021-01-24 19:10:00, 2021-01-24 19:11:00]|350  |
+------------------------------------------+-----+



# Project 3 - Panama Papers

Trace "Spring Song International Co., Ltd." entity with Spark SQL using the following dataset</br>
/dataset/offshore_leaks.nodes.entity.csv </br>
/dataset/offshore_leaks.nodes.intermediary.csv </br>
/dataset/offshore_leaks.edges.csv </br>
/dataset/offshore_leaks.nodes.officer.csv

In [ ]:
from pyspark.sql.functions import *

df_entities = spark \
  .read \
  .format("csv") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/dataset/offshore_leaks.nodes.entity.csv")
  

In [ ]:
df_entities.printSchema()

root
 |-- node_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- jurisdiction: string (nullable = true)
 |-- jurisdiction_description: string (nullable = true)
 |-- country_codes: string (nullable = true)
 |-- countries: string (nullable = true)
 |-- incorporation_date: string (nullable = true)
 |-- inactivation_date: string (nullable = true)
 |-- struck_off_date: string (nullable = true)
 |-- closed_date: string (nullable = true)
 |-- ibcRUC: string (nullable = true)
 |-- status: string (nullable = true)
 |-- company_type: string (nullable = true)
 |-- service_provider: string (nullable = true)
 |-- sourceID: string (nullable = true)
 |-- valid_until: string (nullable = true)
 |-- note: string (nullable = true)



In [ ]:
df_entities.show()

+-------+--------------------+------------+------------------------+-------------+--------------------+------------------+-----------------+---------------+-----------+-------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+
|node_id|                name|jurisdiction|jurisdiction_description|country_codes|           countries|incorporation_date|inactivation_date|struck_off_date|closed_date| ibcRUC|              status|        company_type|    service_provider|      sourceID|         valid_until|                note|
+-------+--------------------+------------+------------------------+-------------+--------------------+------------------+-----------------+---------------+-----------+-------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+
|  67028|          ZODIAK LTD|         XXX|            Undetermined|      VGB;CYP|British Virgin Is...|      

In [ ]:
df_intermediaries = spark \
  .read \
  .format("csv") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/dataset/offshore_leaks.nodes.intermediary.csv")
  

In [ ]:
df_intermediaries.printSchema()

root
 |-- node_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- country_codes: string (nullable = true)
 |-- countries: string (nullable = true)
 |-- status: string (nullable = true)
 |-- sourceID: string (nullable = true)
 |-- valid_until: string (nullable = true)
 |-- note: string (nullable = true)



In [ ]:
df_edges = spark \
  .read \
  .format("csv") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/dataset/offshore_leaks.edges.csv")

In [ ]:
df_edges.printSchema()

root
 |-- START_ID: integer (nullable = true)
 |-- TYPE: string (nullable = true)
 |-- END_ID: integer (nullable = true)
 |-- link: string (nullable = true)
 |-- start_date: string (nullable = true)
 |-- end_date: string (nullable = true)
 |-- sourceID: string (nullable = true)
 |-- valid_until: string (nullable = true)



In [ ]:
df_edges.show()

+--------+------------------+------+------------------+----------+--------+--------------+--------------------+
|START_ID|              TYPE|END_ID|              link|start_date|end_date|      sourceID|         valid_until|
+--------+------------------+------+------------------+----------+--------+--------------+--------------------+
|   67028|registered_address|263908|registered address|      null|    null|Offshore Leaks|The Offshore Leak...|
|   67243|registered_address|172931|registered address|      null|    null|Offshore Leaks|The Offshore Leak...|
|   67258|registered_address|264788|registered address|      null|    null|Offshore Leaks|The Offshore Leak...|
|   67266|registered_address|263908|registered address|      null|    null|Offshore Leaks|The Offshore Leak...|
|  108050|registered_address|285718|registered address|      null|    null|Offshore Leaks|The Offshore Leak...|
|  108051|registered_address|172931|registered address|      null|    null|Offshore Leaks|The Offshore L

In [ ]:
df_officier = spark \
  .read \
  .format("csv") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/dataset/offshore_leaks.nodes.officer.csv")

In [ ]:
df_officier.show()

+-------+--------------------+-------------+--------------------+--------------+--------------------+----+
|node_id|                name|country_codes|           countries|      sourceID|         valid_until|note|
+-------+--------------------+-------------+--------------------+--------------+--------------------+----+
|  51113|Bisbro Investment...|          VGB|British Virgin Is...|Offshore Leaks|The Offshore Leak...|null|
|  51114|Mohamed Mustafa B...|          MYS|            Malaysia|Offshore Leaks|The Offshore Leak...|null|
|  51115|        Teh Hong Eng|          HKG|           Hong Kong|Offshore Leaks|The Offshore Leak...|null|
|  51116|New Sights Enterp...|          VGB|British Virgin Is...|Offshore Leaks|The Offshore Leak...|null|
|  51117|       Yong Meow Mui|          SGP|           Singapore|Offshore Leaks|The Offshore Leak...|null|
|  51118|        CHEN GUOSHUN|          CHN|               China|Offshore Leaks|The Offshore Leak...|null|
|  60877|       Chan Yat Ying|       

In [ ]:
df_entity.show()

+-------+-----------------+------------+------------------------+-------------+--------------------+------------------+-----------------+---------------+-----------+------+------+--------------------+--------------------+--------------+--------------------+----+
|node_id|             name|jurisdiction|jurisdiction_description|country_codes|           countries|incorporation_date|inactivation_date|struck_off_date|closed_date|ibcRUC|status|        company_type|    service_provider|      sourceID|         valid_until|note|
+-------+-----------------+------------+------------------------+-------------+--------------------+------------------+-----------------+---------------+-----------+------+------+--------------------+--------------------+--------------+--------------------+----+
| 109743|Tannehill Limited|         XXX|            Undetermined|      VGB;SGP|British Virgin Is...|       12-JUL-2004|             null|           null|       null|605746|Active|Standard Company ...|Commonwealt

In [ ]:
df_nodes = (df_entities.filter(col("name") == "Spring Song International Co., Ltd.")
            .join(df_edges, df_edges.END_ID == df_entities.node_id)            
            .join(df_intermediaries, df_edges.START_ID == df_intermediaries.node_id)            
            )

In [ ]:
df_nodes.show(truncate = False)

+-------+-----------------------------------+------------+------------------------+-------------+----------------------+------------------+-----------------+---------------+-----------+-------+------+----------------------------------+-------------------+--------------+-----------------------------------------------+----+--------+---------------+------+----------------------+----------+--------+--------------+-----------------------------------------------+-------+---------------------------------+---------------+---------------------------------------------------+------+--------------+-----------------------------------------------+----+
|node_id|name                               |jurisdiction|jurisdiction_description|country_codes|countries             |incorporation_date|inactivation_date|struck_off_date|closed_date|ibcRUC |status|company_type                      |service_provider   |sourceID      |valid_until                                    |note|START_ID|TYPE           |END_I